In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Reshape
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Lambda, Activation
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

In [2]:
def conv_bn_pool(
    inp_tensor,
    layer_idx,
    conv_filters,
    conv_kernel_size,
    conv_strides,
    conv_pad,
    pool = '',
    pool_size = (2, 2),
    pool_strides = None,
    conv_layer_prefix = 'conv',
):
    x = ZeroPadding2D(padding = conv_pad, name = 'pad{}'.format(layer_idx))(
        inp_tensor
    )
    x = Conv2D(
        filters = conv_filters,
        kernel_size = conv_kernel_size,
        strides = conv_strides,
        padding = 'valid',
        name = '{}{}'.format(conv_layer_prefix, layer_idx),
    )(x)
    x = BatchNormalization(
        epsilon = 1e-5, momentum = 1.0, name = 'bn{}'.format(layer_idx)
    )(x)
    x = Activation('relu', name = 'relu{}'.format(layer_idx))(x)
    if pool == 'max':
        x = MaxPooling2D(
            pool_size = pool_size,
            strides = pool_strides,
            name = 'mpool{}'.format(layer_idx),
        )(x)
    elif pool == 'avg':
        x = AveragePooling2D(
            pool_size = pool_size,
            strides = pool_strides,
            name = 'apool{}'.format(layer_idx),
        )(x)
    return x


# Block of layers: Conv --> BatchNorm --> ReLU --> Dynamic average pool (fc6 -> apool6 only)
def conv_bn_dynamic_apool(
    inp_tensor,
    layer_idx,
    conv_filters,
    conv_kernel_size,
    conv_strides,
    conv_pad,
    conv_layer_prefix = 'conv',
):
    x = ZeroPadding2D(padding = conv_pad, name = 'pad{}'.format(layer_idx))(
        inp_tensor
    )
    x = Conv2D(
        filters = conv_filters,
        kernel_size = conv_kernel_size,
        strides = conv_strides,
        padding = 'valid',
        name = '{}{}'.format(conv_layer_prefix, layer_idx),
    )(x)
    x = BatchNormalization(
        epsilon = 1e-5, momentum = 1.0, name = 'bn{}'.format(layer_idx)
    )(x)
    x = Activation('relu', name = 'relu{}'.format(layer_idx))(x)
    x = GlobalAveragePooling2D(name = 'gapool{}'.format(layer_idx))(x)
    x = Reshape((1, 1, conv_filters), name = 'reshape{}'.format(layer_idx))(x)
    return x

In [3]:
class Resnet1D(Model):
    def __init__(self, params=None, is_training=False):
        super(Resnet1D, self).__init__()
    
    def call(self, inputs, training=None, mask=None):
        inp = inputs['features_input']
        x = conv_bn_pool(
            inp,
            layer_idx = 1,
            conv_filters = 96,
            conv_kernel_size = (7, 7),
            conv_strides = (2, 2),
            conv_pad = (1, 1),
            pool = 'max',
            pool_size = (3, 3),
            pool_strides = (2, 2),
        )
        x = conv_bn_pool(
            x,
            layer_idx = 2,
            conv_filters = 256,
            conv_kernel_size = (5, 5),
            conv_strides = (2, 2),
            conv_pad = (1, 1),
            pool = 'max',
            pool_size = (3, 3),
            pool_strides = (2, 2),
        )
        x = conv_bn_pool(
            x,
            layer_idx = 3,
            conv_filters = 384,
            conv_kernel_size = (3, 3),
            conv_strides = (1, 1),
            conv_pad = (1, 1),
        )
        x = conv_bn_pool(
            x,
            layer_idx = 4,
            conv_filters = 256,
            conv_kernel_size = (3, 3),
            conv_strides = (1, 1),
            conv_pad = (1, 1),
        )
        x = conv_bn_pool(
            x,
            layer_idx = 5,
            conv_filters = 256,
            conv_kernel_size = (3, 3),
            conv_strides = (1, 1),
            conv_pad = (1, 1),
            pool = 'max',
            pool_size = (5, 3),
            pool_strides = (3, 2),
        )
        x = conv_bn_dynamic_apool(
            x,
            layer_idx = 6,
            conv_filters = 4096,
            conv_kernel_size = (9, 1),
            conv_strides = (1, 1),
            conv_pad = (0, 0),
            conv_layer_prefix = 'fc',
        )
        x = conv_bn_pool(
            x,
            layer_idx = 7,
            conv_filters = 1024,
            conv_kernel_size = (1, 1),
            conv_strides = (1, 1),
            conv_pad = (0, 0),
            conv_layer_prefix = 'fc',
        )
        x = Lambda(lambda y: K.l2_normalize(y, axis = 3), name = 'norm')(x)
        x = Conv2D(
            filters = 1024,
            kernel_size = (1, 1),
            strides = (1, 1),
            padding = 'valid',
            name = 'fc8',
        )(x)
        return x
    
class Model:
    def __init__(self, is_training = False):
        self.X = tf.placeholder(tf.float32, [None, 512, None, 1])
        self.model = Resnet1D(is_training = is_training)
        inputs = {'features_input': self.X}
        self.logits = self.model.call(inputs)
        self.logits = self.logits[:,0,0,:]

In [4]:
ckpt_path = 'v1/vggvox.ckpt'

In [5]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()
sess.run(tf.global_variables_initializer())
var_lists = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, ckpt_path)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Restoring parameters from v1/vggvox.ckpt


In [6]:
# Signal processing
SAMPLE_RATE = 16000
PREEMPHASIS_ALPHA = 0.97
FRAME_LEN = 0.025
FRAME_STEP = 0.01
NUM_FFT = 512
BUCKET_STEP = 1
MAX_SEC = 10

# Model
WEIGHTS_FILE = "data/model/weights.h5"
COST_METRIC = "cosine"  # euclidean or cosine
INPUT_SHAPE=(NUM_FFT,None,1)

# IO
ENROLL_LIST_FILE = "cfg/enroll_list.csv"
TEST_LIST_FILE = "cfg/test_list.csv"
RESULT_FILE = "res/results.csv"

In [7]:
import librosa
import numpy as np
from scipy.signal import lfilter, butter

import sigproc


def load_wav(filename, sample_rate):
    audio, sr = librosa.load(filename, sr = sample_rate, mono = True)
    audio = audio.flatten()
    return audio


def normalize_frames(m, epsilon = 1e-12):
    return np.array([(v - np.mean(v)) / max(np.std(v), epsilon) for v in m])


# https://github.com/christianvazquez7/ivector/blob/master/MSRIT/rm_dc_n_dither.m
def remove_dc_and_dither(sin, sample_rate):
    if sample_rate == 16e3:
        alpha = 0.99
    elif sample_rate == 8e3:
        alpha = 0.999
    else:
        print('Sample rate must be 16kHz or 8kHz only')
        exit(1)
    sin = lfilter([1, -1], [1, -alpha], sin)
    dither = (
        np.random.random_sample(len(sin))
        + np.random.random_sample(len(sin))
        - 1
    )
    spow = np.std(dither)
    sout = sin + 1e-6 * spow * dither
    return sout


def get_fft_spectrum(filename, buckets = None):
    signal = load_wav(filename, SAMPLE_RATE)
    signal *= 2 ** 15

    # get FFT spectrum
    signal = remove_dc_and_dither(signal, SAMPLE_RATE)
    signal = sigproc.preemphasis(signal, coeff = PREEMPHASIS_ALPHA)
    frames = sigproc.framesig(
        signal,
        frame_len = FRAME_LEN * SAMPLE_RATE,
        frame_step = FRAME_STEP * SAMPLE_RATE,
        winfunc = np.hamming,
    )
    fft = abs(np.fft.fft(frames, n = NUM_FFT))
    fft_norm = normalize_frames(fft.T)
    
    if buckets:

        # truncate to max bucket sizes
        rsize = max(k for k in buckets if k <= fft_norm.shape[1])
        rstart = int((fft_norm.shape[1] - rsize) / 2)
        out = fft_norm[:, rstart : rstart + rsize]
        return out
    
    else:
        return fft_norm

In [8]:
def build_buckets(max_sec, step_sec, frame_step):
    buckets = {}
    frames_per_sec = int(1 / frame_step)
    end_frame = int(max_sec * frames_per_sec)
    step_frame = int(step_sec * frames_per_sec)
    for i in range(0, end_frame + 1, step_frame):
        s = i
        s = np.floor((s - 7 + 2) / 2) + 1  # conv1
        s = np.floor((s - 3) / 2) + 1  # mpool1
        s = np.floor((s - 5 + 2) / 2) + 1  # conv2
        s = np.floor((s - 3) / 2) + 1  # mpool2
        s = np.floor((s - 3 + 2) / 1) + 1  # conv3
        s = np.floor((s - 3 + 2) / 1) + 1  # conv4
        s = np.floor((s - 3 + 2) / 1) + 1  # conv5
        s = np.floor((s - 3) / 2) + 1  # mpool5
        s = np.floor((s - 1) / 1) + 1  # fc6
        if s > 0:
            buckets[i] = int(s)
    return buckets

buckets = build_buckets(MAX_SEC, BUCKET_STEP, FRAME_STEP)
buckets

{100: 2,
 200: 5,
 300: 8,
 400: 11,
 500: 14,
 600: 17,
 700: 20,
 800: 23,
 900: 27,
 1000: 30}

In [10]:
from glob import glob

files = glob('data/wav/enroll/*.wav')
files

['data/wav/enroll/27-123349-0000.wav',
 'data/wav/enroll/19-227-0000.wav',
 'data/wav/enroll/26-495-0000.wav']

In [13]:
def pred(x):
    r = sess.run(model.logits, feed_dict = {model.X: np.expand_dims([x], -1)})
    return r

In [14]:
pred(get_fft_spectrum(files[0], buckets)).shape

(1, 1024)

In [15]:
import random

files = glob('../voxceleb/aac/**/*.m4a', recursive = True)
files = random.sample(files, 10000)
len(files)

10000

In [16]:
!rm -rf test-*.wav pickle-*.pkl

In [17]:
import pickle
import mp
from tqdm import tqdm
from pydub import AudioSegment

def loop(args):
    files = args[0]
    index = args[1]
    results = []
    for file in tqdm(files):
        
        audio = AudioSegment.from_file(file[1])
        audio.set_frame_rate(16000).set_channels(1).export(f'test-{index}.wav', format="wav")
        l = get_fft_spectrum(f'test-{index}.wav', buckets)
        results.append((file[0], file[1], l))
        
    with open(f'pickle-{index}.pkl', 'wb') as fopen:
        pickle.dump(results, fopen)

files_index = [(no, f) for no, f in enumerate(files)]
r = mp.multiprocessing(files_index, loop, cores = 50)

100%|██████████| 200/200 [01:58<00:00,  1.68it/s]

100%|█████████▉| 199/200 [01:59<00:00,  1.50it/s]

100%|██████████| 200/200 [01:59<00:00,  1.67it/s]

100%|██████████| 200/200 [02:01<00:00,  1.28it/s]

100%|██████████| 200/200 [02:02<00:00,  1.63it/s]


TypeError: 'NoneType' object is not iterable

In [18]:
!rm -rf test-*.wav

In [19]:
import pandas as pd

df = pd.read_csv('../voxceleb/vox2_meta.csv')
df = df[df['Set '] == 'test ']
speakers = df['VoxCeleb2 ID '].unique().tolist()
speakers = [s.strip() for s in speakers]

In [20]:
from collections import defaultdict

speakers_idx = defaultdict(list)

for speaker in speakers:
    for file in files:
        if speaker in file:
            speakers_idx[speaker].append(file)

In [21]:
from tqdm import tqdm


k = 10
labels = []

def get_id(file):
    return file.split('/')[3]

for file in tqdm(files):
    left_speaker = get_id(file)
    for speaker in speakers:
        if left_speaker == speaker:
            label = 1
        else:
            label = 0
        samples = random.sample(speakers_idx[speaker], min(k, len(speakers_idx[speaker])))
        for s in samples:
            labels.append((label, file, s))

100%|██████████| 10000/10000 [00:18<00:00, 548.35it/s]


In [22]:
random.shuffle(labels)

In [23]:
import itertools
import pickle

pickles = glob('pickle-*.pkl')

pooled = []
for p in pickles:
    with open(p, 'rb') as fopen:
        pooled.append(pickle.load(fopen))
        
pooled = list(itertools.chain(*pooled))

In [24]:
mapping = {i[1]: pred(i[2]) for i in tqdm(pooled)}

100%|██████████| 10000/10000 [00:47<00:00, 210.48it/s]


In [25]:
scores, ls = [], []

for i in tqdm(range(len(labels))):
    ls.append(labels[i][0])
    scores.append(np.sum(mapping[labels[i][1]][0] * mapping[labels[i][2]][0]))

100%|██████████| 11720000/11720000 [02:07<00:00, 91983.96it/s] 


In [26]:
def calculate_eer(y, y_score):
    
    from scipy.optimize import brentq
    from sklearn.metrics import roc_curve
    from scipy.interpolate import interp1d

    fpr, tpr, thresholds = roc_curve(y, y_score, pos_label=1)
    eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    thresh = interp1d(fpr, thresholds)(eer)
    return eer, thresh

In [27]:
calculate_eer(ls, scores)

(0.14070736036804815, array(0.18324658))